In [48]:
#importing
import pandas as pd
import numpy as np
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error
train_raw = pd.read_csv("train.csv")

In [49]:
#Data Cleaning & Preperation 
numericColumns = train_raw.select_dtypes([np.number]).dropna() #Select_dtypes filters columns that meet certain condition. np.number returns list of numeric columns
nonNumeric = train_raw.drop(numericColumns, axis = 1)



In [50]:
train.corr()

#Selected every column which has r^2 w/ SalesPrice > 0.3
numericFeaturesSelected = numericColumns[["OverallQual", "YearBuilt", "YearRemodAdd", "MasVnrArea", "LotArea", "LotFrontage", "BsmtFinSF1", "1stFlrSF", "TotalBsmtSF", "2ndFlrSF", "GrLivArea", "TotRmsAbvGrd", "GarageArea", "WoodDeckSF", "OpenPorchSF", "SalePrice"]]

#Selected columns which sounded potnetially meannigful
nonnumericFeaturesSelected = nonNumeric[["Neighborhood", "SaleCondition"]]

#Adding Dummies
nonnumericDummiesOnly = pd.get_dummies(nonnumericFeaturesSelected, prefix = "Dummy")
nonnumericDummiesOnly = nonnumericDummiesOnly.select_dtypes([np.number])

#Creating new df with signifcant explantory variables
train = pd.concat([numericFeaturesSelected, nonnumericDummiesOnly], axis = 1)

In [51]:
#Cleaning (Initial row ct 1460)
train = train.dropna() #Removed null values (Row ct 1121)

In [52]:
y = train["SalePrice"]
x = train.drop(["SalePrice"], axis = 1) #Needs to be in list, axis = 1 look for columns, default axis =0 looks for rows 
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 2)


In [53]:
#Regression 
lin = LinearRegression()
lin.fit(xTrain, yTrain)

print(lin.coef_)

print(r2_score(lin.predict(xTrain), yTrain))

print(r2_score(lin.predict(xTest), yTest))

#This linear model ;ppls solid. Lets see how machine learning does 

[ 1.36756902e+04  2.41312047e+02  3.15355727e+02  1.77476248e+01
  9.50756076e-01  1.68245156e+02  2.55192051e+01  6.92294914e+01
  2.01895960e+01  6.94952211e+01 -1.99686283e+01  4.68494257e+02
  2.15970383e+01  1.49698212e+01 -1.06116702e+01 -1.54653301e+04
 -4.38261992e+02 -1.68217206e+04  8.96787092e+03  2.36199082e+03
 -7.33321314e+03  2.01221313e+04 -1.80936100e+04 -1.13862675e+04
 -4.62186935e+00 -8.41215730e+03 -1.70864631e+04 -8.88924483e+03
 -4.44645268e+03 -1.75967128e+04  3.53867883e+04  2.94465223e+04
 -1.06508593e+04 -5.47439252e+03 -1.00519941e+04 -1.14948891e+04
  2.04633553e+03  4.26539296e+04 -7.60639089e+02  2.34212611e+04
 -8.73489446e+03  3.13813737e+04 -1.28756507e+04 -2.47314736e+04
 -2.83034759e+03  1.77909927e+04]
0.8362522780892871
0.6343147054107763


In [54]:
nonnumericDummiesOnly = pd.get_dummies(nonNumeric, prefix = "Dummy")
nonnumericDummiesOnly = nonnumericDummiesOnly.select_dtypes([np.number])
nonnumericDummiesOnly.head()

,Dummy_C (all),Dummy_FV,Dummy_RH,Dummy_RL,Dummy_RM,Dummy_Grvl,Dummy_Pave,Dummy_Grvl,Dummy_Pave,Dummy_IR1,...,Dummy_ConLw,Dummy_New,Dummy_Oth,Dummy_WD,Dummy_Abnorml,Dummy_AdjLand,Dummy_Alloca,Dummy_Family,Dummy_Normal,Dummy_Partial
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0


In [55]:
#Decision Tree

from sklearn import tree

nonnumericDummiesOnly = pd.get_dummies(nonNumeric, prefix = "Dummy")
nonnumericDummiesOnly = nonnumericDummiesOnly.select_dtypes([np.number])


dtTraining = pd.concat([numericColumns, nonnumericDummiesOnly], axis = 1)
dtTraining = dtTraining.dropna()

y = dtTraining["SalePrice"]
x = dtTraining.drop(["SalePrice"], axis = 1)
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 2)



dtr = tree.DecisionTreeRegressor(max_depth=10, min_samples_leaf=3)
dtr.fit(xTrain, yTrain)
dTreePrediction = dtr.predict(xTest)

print("R^2:")
print(r2_score(dTreePrediction, yTest))

#A basic Decision Tree does no better than a manual regression. Lets try a random forest

R^2:
0.5896412257028281


In [56]:
#Random Forest

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(xTrain, yTrain)
rForestPrediction = rfr.predict(xTest)

print("R^2:")
print(r2_score(rForestPrediction, yTest))


#The Random Forest Model appears to be the strongest!


R^2:
0.7907844105912085


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
